In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")  // Runs on local machine with 1 local worker / thread, so we can see the impact on COPY_ON_WRITE
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

import spark.implicits._

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@1fcbeb4c
import spark.implicits._

In [3]:
case class Employee(id:Int, name:String, department:String, managerId:Option[Int])
val data = Seq(
  Employee(101, "John", "A", None), 
  Employee(102, "Dan", "A", Some(101)), 
  Employee(103, "James", "A", Some(101)), 
  Employee(104, "Amy", "A", Some(101)), 
  Employee(105, "Anne", "A", Some(101)), 
  Employee(106, "Ron", "B", Some(101))
)

val employeeDs : Dataset[Employee] = data.toDS()
employeeDs.show

+---+-----+----------+---------+
| id| name|department|managerId|
+---+-----+----------+---------+
|101| John|         A|     null|
|102|  Dan|         A|      101|
|103|James|         A|      101|
|104|  Amy|         A|      101|
|105| Anne|         A|      101|
|106|  Ron|         B|      101|
+---+-----+----------+---------+



defined class Employee
data: Seq[Employee] = List(
  Employee(id = 101, name = "John", department = "A", managerId = None),
  Employee(
    id = 102,
    name = "Dan",
    department = "A",
    managerId = Some(value = 101)
  ),
  Employee(
    id = 103,
    name = "James",
    department = "A",
    managerId = Some(value = 101)
  ),
  Employee(
    id = 104,
    name = "Amy",
    department = "A",
    managerId = Some(value = 101)
  ),
  Employee(
    id = 105,
    name = "Anne",
    department = "A",
    managerId = Some(value = 101)
  ),
  Employee(
    id = 106,
    name = "Ron",
    department = "B",
    managerId = Some(value = 101)
  )
)
employeeDs: Dataset[Employee] = [id: int, name: string ... 2 more fields]

Write a solution to find managers with at least five direct reports.

Return the result table in any order.

The result format is in the following example.
| name |
|------|
| John |

In [25]:
val managerCountDF = employeeDs.groupBy(col("managerId")).agg(count(lit(1)).alias("cnt")).filter(col("cnt") >= 5)
managerCountDF.as("a").join(employeeDs.select(col("id"), col("name")).as("b"), col("b.id") === col("a.managerId"), "leftouter").select(col("name")).show()

+----+
|name|
+----+
|John|
+----+



managerCountDF: Dataset[Row] = [managerId: int, cnt: bigint]

In [ ]:
spark.stop()